# 시금치 & 오이 가격 예측 모델 (수출입 데이터 포함)
*기존 데이터에 수출입 중량 데이터를 추가하고, 다양한 모델과 하이퍼파라미터를 테스트하여 종합적인 평가 지표(MAE, MSE, RMSE, R2)를 통해 최적의 모델을 선정합니다.*

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

## 1. 데이터 불러오기 및 통합 전처리

In [ ]:
# 데이터 불러오기
cold_df = pd.read_excel('../data//mon_cold.xlsx')
wind_df = pd.read_excel('../data//mon_wind.xlsx')
hot_df = pd.read_excel('../data//mon_hot.xlsx')
price_df = pd.read_excel('../data//region_price.xlsx')
weather_df = pd.read_csv('../data//region_weather.csv')
trade_df = pd.read_excel('../data/spinach_cucumber_df.xlsx')

# 데이터 변환 (long format)
cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')

# 날짜 형식 통일 (YYYY-MM)
for df in [price_df, weather_df, trade_df]:
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce').dt.strftime('%Y-%m')

# 데이터 병합 (가격을 중심으로)
merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, trade_df, on=['품목', '날짜'], how='left')

# 데이터 정제
merged_df.dropna(subset=['평균가격'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()
merged_df.fillna(0, inplace=True)

# 특성 공학
merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month
print('데이터 통합 및 전처리 완료. 최종 데이터셋 일부:')
print(merged_df.head())

데이터 통합 및 전처리 완료. 최종 데이터셋 일부:
        지역         날짜  품목          평균가격      총거래물량  평균기온(°C)  \
0      강원도 2021-01-01  감자  24245.289659  3012880.0 -3.435714   
1      경기도 2021-01-01  감자  24006.188904   356360.0 -3.680000   
2  제주특별자치도 2021-01-01  감자  37886.008230   330480.0  6.700000   
3     경상남도 2021-01-01  감자  27559.604013   533300.0  1.192857   
4     경상북도 2021-01-01  감자  24701.552614   462160.0 -1.421429   

   월합강수량(00~24h만)(mm)  평균풍속(m/s)  최심적설(cm)  한파발생  태풍발생  폭염발생  수입중량  수출중량  \
0           10.221429   1.778571       4.9   1.0   0.0   0.0   0.0   0.0   
1           18.460000   1.440000       9.4   1.0   0.0   0.0   0.0   0.0   
2           84.225000   4.575000       9.2   0.0   0.0   0.0   0.0   0.0   
3           17.678571   1.700000       0.0   1.0   0.0   0.0   0.0   0.0   
4           12.728571   2.350000       0.9   1.0   0.0   0.0   0.0   0.0   

     연도  월  
0  2021  1  
1  2021  1  
2  2021  1  
3  2021  1  
4  2021  1  


## 2. 모델 학습, 튜닝 및 종합 평가

In [9]:
def find_best_model_with_full_evaluation(df, vegetable_name):
    target_df = df[df['품목'] == vegetable_name].copy()
    target_df['지역'] = target_df['지역'].astype('category').cat.codes

    # **수출입 중량 특성 추가**
    features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '지역', '연도', '월', '수출중량', '수입중량']
    target = '평균가격'

    X = target_df[features]
    y = target_df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 및 파라미터 그리드 정의
    models_and_params = [
        ('RandomForest', RandomForestRegressor(random_state=42), {'n_estimators': [100, 200], 'max_depth': [5, 10, 20]}),
        ('XGBoost', XGBRegressor(random_state=42), {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1]}),
        ('LightGBM', LGBMRegressor(random_state=42, verbose=-1), {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1]})
    ]

    best_estimators = []
    results_summary = []

    print(f'--- {vegetable_name} 모델 튜닝 및 평가 시작 ---')

    for name, model, params in models_and_params:
        print(f'{name} 모델 튜닝 중...')
        grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, n_jobs=-1, scoring='r2')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        # 평가 지표 계산
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        results_summary.append({'Model': name, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2, 'Best Params': grid_search.best_params_})
        best_estimators.append((name, best_model))

    # 앙상블 모델 평가
    print('앙상블(Voting) 모델 평가 중...')
    voting_model = VotingRegressor(estimators=best_estimators)
    voting_model.fit(X_train, y_train)
    y_pred_voting = voting_model.predict(X_test)
    mae_v = mean_absolute_error(y_test, y_pred_voting)
    mse_v = mean_squared_error(y_test, y_pred_voting)
    rmse_v = np.sqrt(mse_v)
    r2_v = r2_score(y_test, y_pred_voting)
    results_summary.append({'Model': 'VotingEnsemble', 'MAE': mae_v, 'MSE': mse_v, 'RMSE': rmse_v, 'R2': r2_v, 'Best Params': 'N/A'})

    # 최종 결과 출력
    results_df = pd.DataFrame(results_summary).sort_values(by='R2', ascending=False)
    best_model_info = results_df.iloc[0]

    print(f'--- {vegetable_name} 최종 모델 평가 결과 ---')
    print(results_df.to_string())
    print(f'=> {vegetable_name}의 최종 최적 모델은 {best_model_info["Model"]} 입니다.')
    print(f'   - R2: {best_model_info["R2"]:.4f}')
    print(f'   - 최적 파라미터: {best_model_info["Best Params"]}')
    print('-'*70)

# 함수 실행
find_best_model_with_full_evaluation(merged_df, '시금치')
find_best_model_with_full_evaluation(merged_df, '오이')

--- 시금치 모델 튜닝 및 평가 시작 ---
RandomForest 모델 튜닝 중...
XGBoost 모델 튜닝 중...
LightGBM 모델 튜닝 중...
앙상블(Voting) 모델 평가 중...
--- 시금치 최종 모델 평가 결과 ---
            Model          MAE           MSE         RMSE        R2                                                   Best Params
3  VotingEnsemble  2520.730890  3.249497e+07  5700.435882  0.721114                                                           N/A
0    RandomForest  2748.606547  3.286898e+07  5733.147897  0.717904                        {'max_depth': 20, 'n_estimators': 100}
2        LightGBM  2736.081603  3.333075e+07  5773.279368  0.713941   {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
1         XGBoost  2639.344366  3.581256e+07  5984.359488  0.692641  {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}
=> 시금치의 최종 최적 모델은 VotingEnsemble 입니다.
   - R2: 0.7211
   - 최적 파라미터: N/A
----------------------------------------------------------------------
--- 오이 모델 튜닝 및 평가 시작 ---
RandomForest 모델 튜닝 중...
XGBoost 모델 튜닝 중...
Lig